# R.O.P: Example shellCommand

This is a reimplementation of fast.ai" shellCommand with ROP.
The goal is th be "aware" of **good** and **bad** result, with details.



## TL;DR
This complete example shows how to use ROP to create a "shellCommand" utility funciton that is aware of bot errors (misspelled commands and exceptions) and command result (standard unix convention where:'0' means OK).

```swift
func countNumberOfFilesInCurrentFolder() -> Result<Int> {
    return shellCommandROP("/bin/pwd")
        .then{pwd in pwd.replacingOccurrences(of: "\n", with: "")} // Use previous result 
        .then{pwd in shellCommandROP("/bin/ls", [pwd])} // Use the clean string to search files
        .then(myLinesCounter)
}
let complexResult = countNumberOfFilesInCurrentFolder(); 
printResult("The count of number of files in ls is:", complexResult)
```
> The count of number of files in ls is: 3

## ROP: everything is there

In [1]:
//Export
protocol DescriptiveError {
    func errorMessage()->String
}

struct SimpleError: DescriptiveError {
    let msg: String // Immutable
    init(_ msg: String) { self.msg = msg }
    func errorMessage() -> String { return msg }
}

enum Result<OK> {
    case Ok(OK)
    case Error(DescriptiveError) // We always want an error that follows this protocol
    
    // Methods and Properties
    var value: OK? {
        get {
            switch self {
                case .Ok(let value): return value
                case .Error(_): return nil // Don't need the value
            }
        }
    }    
    var error: String? {
        get {
            switch self {
                case .Ok(_): return nil
                case .Error(let err): return err.errorMessage()
            }
        }
    }
    //MAP: spark of magic..
    func then<OUT>(_ f: (OK)->OUT ) -> Result<OUT> {
        switch self {
            case .Ok(let val): return Result<OUT>.Ok(f(val))
            case .Error(let err): return Result<OUT>.Error(err) // Forward error
        }
    }
    //FLAT MAT: the real magic of monad! 
    func then<OUT>(_ f: (OK)->Result<OUT> ) -> Result<OUT> {
        switch self {
            case .Ok(let val): return f(val)
            case .Error(let err): return Result<OUT>.Error(err) // Forward error
        }
    }
    //TAP: result snooping 
    func use(_ f: (OK)->() ) -> Result<OK> {
        switch self {
            case .Ok(let val): f(val)
            case .Error(let err): print("ERROR HAS HAPPENED: \(err.errorMessage())")
        }
        return self
    }
    //Convenience
    func isOk() -> Bool {
        switch self {
            case .Ok: return true
            case .Error: return false
        }
    }
}

//Convenience factory functions as static to quick find them with auto completition...
func good<T>(_ value: T) -> Result<T> {
    return .Ok(value) // Thnx Chris ;-)
}
func bad<T>(_ msg: String) -> Result<T> {
    return .Error(SimpleError(msg)) // Thnx Chris ;-)
}
//Create bad result from generic DescriptiveError
func bad<T>(_ err: DescriptiveError) -> Result<T> {
    return Result<T>.Error(err)
}
//Forwards errors adapting type to destination...
func bad<T,P>(_ parent: Result<P>) -> Result<T> {
    switch parent {
        case .Ok: return bad("WARNING: parent was not bad!") // good goes bad!! Warning!!
        case .Error(let val): return Result<T>.Error(val) // bad remains bad...
    }
}

# Shell Command with ROP

In [2]:
//export
struct ShellCommandError: DescriptiveError {
    let code: Int32 // Immutable
    let msg: String // Immutable
    init(_ code: Int32, _ msg: String) { 
        self.code = code
        self.msg = msg 
    }
    func errorMessage() -> String { return "[\(code)]: \(msg)" }
}

In [3]:
//export
import Foundation

In [4]:
//export
@discardableResult
public func shellCommandROP(_ launchPath: String, _ arguments: [String] = []) -> Result<String>
{
    let task = Process()
    task.executableURL = URL(fileURLWithPath: launchPath)
    task.arguments = arguments

    let pipe = Pipe(); task.standardOutput = pipe
    let errPipe = Pipe(); task.standardError = errPipe
    do {
        try task.run()
        task.waitUntilExit()
        let data = pipe.fileHandleForReading.readDataToEndOfFile()
        let stdErr = errPipe.fileHandleForReading.readDataToEndOfFile()
        let text = String(data: data, encoding: String.Encoding.utf8) ?? ""
        let err = String(data: stdErr, encoding: String.Encoding.utf8) ?? ""
        if 0==task.terminationStatus {
            return good(text)
        } else {
            // Don't use the standard "bad" result, but keep termination status...
            return bad(ShellCommandError(task.terminationStatus, err))
        }
    } catch {
        return bad("ERROR: \(error).") // Incredible, to convert it I've just changed print to "bad"
    }
}

**IMPORTANT**: Now our shellCommand is "aware of errors". So all the time we call it we're going to understand if something **good** or **bad** has happened and **why** it happened.

In [5]:
let resultOfGoodShellCommand = shellCommandROP("/bin/ls", ["-lh"])
print("Was last command successful?! Answer: \(resultOfGoodShellCommand.isOk())")

Was last command successful?! Answer: true


In [6]:
let resultOfMispellShellCommand = shellCommandROP("/bin/wtf", ["-lh"]) // wtf is not a command
print("Was last command successful?! Answer: \(resultOfMispellShellCommand.isOk())")

Was last command successful?! Answer: false


In [7]:
let resultOfGoodShellCommandWithBadResult = shellCommandROP("/bin/ls", ["-lh","/wtf"]) // wtf is not a folder
print("Was last command successful?! Answer: \(resultOfGoodShellCommandWithBadResult.isOk())")
print("What's happened? Answer: \(resultOfGoodShellCommandWithBadResult.error!)")

Was last command successful?! Answer: false
What's happened? Answer: [2]: /bin/ls: cannot access '/wtf': No such file or directory



## Try shellCommand

In [8]:
func printResult<OK>(_ msg: String, _ result: Result<OK>) {
    switch result {
        case .Ok(let val): print(msg, val)
        case .Error(let err): print(msg, "ERROR:", err.errorMessage())
    }
}

In [9]:
let theFolder = "/notebooks/swift-rop/jupyter/tmp"
let theFile = theFolder + "/test.txt"
print("1) create the folder \(theFolder)",shellCommandROP("/bin/mkdir",[theFolder]))
print("2) list files in the folder",shellCommandROP("/bin/ls", [theFolder])) 
print("3) create the file \(theFile)",shellCommandROP("/bin/touch", [theFile])) 
print("4) run non existent command",shellCommandROP("/bin/wtf", [theFile])) // wtf is not a command
printResult("5) list files again. Folder content:",shellCommandROP("/bin/ls", ["-lh",theFolder])) // list files in new folder
print("6) remove the file",shellCommandROP("/bin/rm", ["-rf", theFile]))
printResult("7) Folder content:",shellCommandROP("/bin/ls", ["-lh",theFolder])) // list files in new folder
print("8) remove the folder",shellCommandROP("/bin/rmdir",[theFolder]))
print("8) try to list files in deleted folder",shellCommandROP("/bin/ls", [theFolder]))

1) create the folder /notebooks/swift-rop/jupyter/tmp Ok("")
2) list files in the folder Ok("")
3) create the file /notebooks/swift-rop/jupyter/tmp/test.txt Ok("")
4) run non existent command Error(__lldb_expr_15.SimpleError(msg: "ERROR: The operation could not be completed."))
5) list files again. Folder content: total 0
-rw-r--r-- 1 root root 0 Apr 26 16:04 test.txt

6) remove the file Ok("")
7) Folder content: total 0

8) remove the folder Ok("")
8) try to list files in deleted folder Error(__lldb_expr_21.ShellCommandError(code: 2, msg: "/bin/ls: cannot access \'/notebooks/swift-rop/jupyter/tmp\': No such file or directory\n"))


## Let's compose!

In [10]:
// Some support functions
func logString(_ str: String) {
    print("> LOG: '\(str)'")
}

func myLinesCounter(_ str: String) -> Int {
    return str.reduce(into: 0) { (count, ch) in if ch == "\n" { count += 1 } } + 1 
}

//Test
print(myLinesCounter("Theese\nare\nthree lines"))

3


In [11]:
func countNumberOfFilesInFolder(_ folder: String) -> Result<Int> {
    return shellCommandROP("/bin/ls", [folder])
            .then(myLinesCounter)
}
printResult("1) Run on existing folder:", countNumberOfFilesInFolder("/notebooks/swift-rop/jupyter"))
printResult("2) Run on wrong folder: ", countNumberOfFilesInFolder("/wtf"))

1) Run on existing folder: 3
2) Run on wrong folder:  ERROR: [2]: /bin/ls: cannot access '/wtf': No such file or directory



In [12]:
func countNumberOfFilesInCurrentFolder() -> Result<Int> {
    return shellCommandROP("/bin/pwd")
        .then{pwd in pwd.replacingOccurrences(of: "\n", with: "")} // The result of previous command is the input 
        .then{pwd in shellCommandROP("/bin/ls", [pwd])} // Use the clean string to search files
        .then(myLinesCounter)
}
let complexResult = countNumberOfFilesInCurrentFolder(); 
printResult("The count of number of files in ls is:", complexResult)

The count of number of files in ls is: 3


In [13]:
func countNumberOfFilesInCurrentFolderWithLog() -> Result<Int> {
    return shellCommandROP("/bin/pwd")
        .use(logString) // take a look
        .use{_ in print("> Note that there is an extra new line on the result...")} // take a look
        .then{pwd in pwd.replacingOccurrences(of: "\n", with: "")} // Clean pwd output
        .use(logString) // take a look
        .use{_ in print("> Nom more extra lie end: we're done!")} // take a look
        .then{pwd in shellCommandROP("/bin/ls", [pwd])}
        .then(myLinesCounter)
}
printResult("The count of number of files in ls is:", countNumberOfFilesInCurrentFolderWithLog())

> LOG: '/notebooks/swift-rop/jupyter
'
> Note that there is an extra new line on the result...
> LOG: '/notebooks/swift-rop/jupyter'
> Nom more extra lie end: we're done!
The count of number of files in ls is: 3
